## Configure tesseract

In [61]:
import pytesseract
print(pytesseract.get_tesseract_version())
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

5.5.0.20241111


## โหลดรูปเบื้องต้น

In [66]:
import cv2
from matplotlib import table
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

output_folder = Path("../data/output_images/output_V6_TN_TSR")
output_folder.mkdir(exist_ok=True)

image = Image.open("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/good/transcript_tn_f_1.png")
#image = Image.open("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/good/transcript_tn_f_3.png")
#image = Image.open("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/good/transcript_tn_f_4.png")
#image = Image.open("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/good/transcript_tn_f_5.png")
#image = Image.open("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/good/transcript_tn_f_6.png")
#image = Image.open("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/good/transcript_tn_f_7.png")
#image = Image.open("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/good/transcript_tn_f_8.png") # ซ้ำกับ 7
#image = Image.open("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/good/transcript_tn_f_9.png") # จับตารางได้แต่พังตอน pt
#image = Image.open("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/good/transcript_tn_f_10.png") # ลบเส้นตารางไม่หมดตรงรหัสวิชา
#image = Image.open("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/good/transcript_tn_f_11.png") # ภาพใหญ่เกินไป
#image = Image.open("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/good/transcript_tn_f_12.png") # ตารางไม่ตรง
#image = Image.open("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/good/transcript_tn_f_13.png") # ติดลอยปั้มหมึก
#image = Image.open("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/good/transcript_tn_f_14.png")

if image is None:
    raise FileNotFoundError("ไม่พบไฟล์ภาพ กรุณาตรวจสอบเส้นทางของไฟล์")

new_size = (1660, 2347)  # ตัวอย่างขนาดใหม่
resized_pil = image.resize(new_size, Image.LANCZOS) # ปรับขนาดภาพด้วย LANCZOS filter

# แปลงภาพจาก PIL Image เป็น NumPy array (ในรูปแบบ RGB)
img_rgb = np.array(resized_pil)

# แปลงจาก RGB เป็น BGR เพื่อให้ใช้งานกับ OpenCV ได้
img_cv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)

denoised = cv2.bilateralFilter(img_cv, d=9, sigmaColor=75, sigmaSpace=75) # จำกัด noise
gray_img = cv2.cvtColor(denoised, cv2.COLOR_BGR2GRAY)


binary_gaussian = cv2.adaptiveThreshold(
    gray_img, 
    maxValue=255, 
    adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    thresholdType=cv2.THRESH_BINARY_INV, 
    blockSize=51, 
    C=15 #21
)

# สร้าง kernel สำหรับ morphological operation
kernel = np.ones((3, 3), np.uint8)
dilated = cv2.dilate(binary_gaussian, kernel, iterations=1)
# ใช้ closing เพื่อเติมเต็มส่วนที่ขาดของเส้น
closed_dummy = cv2.morphologyEx(binary_gaussian, cv2.MORPH_CLOSE, kernel, iterations=1)

cv2.imwrite(f"{output_folder}/img_cv.png", img_cv)
cv2.imwrite(f"{output_folder}/denoised.png", denoised)
cv2.imwrite(f"{output_folder}/gray.png", gray_img)
cv2.imwrite(f"{output_folder}/binary_g.png", binary_gaussian)
cv2.imwrite(f"{output_folder}/dilated.png", dilated)
cv2.imwrite(f"{output_folder}/closed_dummy.png", closed_dummy)


True

## แยกตารางเกรดกับข้อมูลนักศึกษา

In [67]:
def split_grade_table_and_students(binary_img, denoised, dummy):
    
    # แยกตาราง
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(dummy, connectivity=8)
    areas = [stat[4] for stat in stats]  # ดึงค่า area
    sorted_areas = sorted(areas, reverse=True)  # เรียงลำดับจากมากไปน้อย
    second_max_area = sorted_areas[1]  # ค่าอันดับ 2
    second_max_area_index = areas.index(second_max_area)  # หาตำแหน่งในลิสต์เดิม
    table_position = stats[second_max_area_index]
    x, y, w, h, area = table_position

    table_img = binary_img[y:y+h, x:x+w]
    table_dummy_img = dummy[y:y+h, x:x+w]
    table_original_img = denoised[y:y+h, x:x+w]

    # ข้อมูลนักเรียน
    #x_start = int((x+w) * 0.40) # ความกว้าง 40% ของตาราง
    x_end = int((x+w) * 0.85) # ความกว้าง 85% ของตาราง
    x_split_half = int((x+w) * 0.38) # ความกว้าง 38% ของตาราง

    student_info_img = binary_img[:y, :x_end]
    student_info_fh_img = binary_img[:y, :x_split_half] # ครึ่งแรก
    student_info_sh_img = binary_img[:y, x_split_half:x_end] # ครึ่งหลัง

    return table_img, table_dummy_img, table_original_img, student_info_img, student_info_fh_img, student_info_sh_img

def biggest_contour(contours):
    biggest = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        #print(area)
        if area > 1000:
            #print("มา")
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02 * peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area

    return biggest

def persective_transformation(table_binary_img, table_original_img, table_dummy_img):

    # ค้นหาคอนทัวร์
    contours, hierarchy = cv2.findContours(table_dummy_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #contours, hierarchy = cv2.findContours(table_binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    # ค้นหาสี่เหลี่ยมที่ใหญ่ที่สุด
    biggest = biggest_contour(contours)

    points = biggest.reshape(4, 2)
    input_points = np.zeros((4, 2), dtype="float32")

    points_sum = points.sum(axis=1)
    input_points[0] = points[np.argmin(points_sum)]
    input_points[3] = points[np.argmax(points_sum)]

    points_diff = np.diff(points, axis=1)
    input_points[1] = points[np.argmin(points_diff)]
    input_points[2] = points[np.argmax(points_diff)]

    (top_left, top_right, bottom_right, bottom_left) = input_points

    # Euclidean Distance Formula
    bottom_width = np.sqrt(((bottom_right[0] - bottom_left[0]) ** 2) + ((bottom_right[1] - bottom_left[1]) ** 2))
    top_width = np.sqrt(((top_right[0] - top_left[0]) ** 2) + ((top_right[1] - top_left[1]) ** 2))
    rigth_height = np.sqrt(((top_left[0] - bottom_right[0]) ** 2) + ((top_left[1] - bottom_right[1]) ** 2))
    left_height = np.sqrt(((top_left[0] - bottom_left[0]) ** 2) + ((top_left[1] - bottom_left[1]) ** 2))

    # Output image size
    #max_width = max(int(bottom_width), int(top_width))
    expand_width = round(max(int(bottom_width), int(top_width)) * 0.4)
    max_width = max(int(bottom_width), int(top_width)) + expand_width
    max_height = max(int(rigth_height), int(left_height))

    # Desird points values in the output image
    converted_points = np.float32([[0, 0], [max_width, 0], [0, max_height], [max_width, max_height]])

    # Perspective transformaxtion
    matrix = cv2.getPerspectiveTransform(input_points, converted_points)
    img_out = cv2.warpPerspective(table_binary_img.copy(), matrix, (max_width, max_height))
    img_original_out = cv2.warpPerspective(table_original_img.copy(), matrix, (max_width, max_height))
    img_dummy_out = cv2.warpPerspective(table_dummy_img.copy(), matrix, (max_width, max_height))

    #return img_out, img_original_out
    return img_out, img_original_out, img_dummy_out

table_img, table_dummy_img, table_original_img, student_info_img, student_info_fh_img, student_info_sh_img = split_grade_table_and_students(binary_gaussian, denoised, dilated)
table_persective_img, table_original_persective_img, table_dummy_persective_img = persective_transformation(binary_gaussian, denoised, dilated)
#grid_img = create_grid_image(table_persective_img)
#line_mask, table_without_lines, table_without_lines_2 = hough_line_transform(table_persective_img, table_original_persective_img, grid_img)

#cell_contours = cells_detect(line_mask)
#cell_contours = cells_detect(grid_img)

cv2.imwrite(f"{output_folder}/table_img.png", table_img)
cv2.imwrite(f"{output_folder}/table_dummy_img.png", table_dummy_img)
cv2.imwrite(f"{output_folder}/table_original_img.png", table_original_img)
cv2.imwrite(f"{output_folder}/student_info_img.png", student_info_img)
cv2.imwrite(f"{output_folder}/student_info_fh_img.png", student_info_fh_img)
cv2.imwrite(f"{output_folder}/student_info_sh_img.png", student_info_sh_img)

cv2.imwrite(f"{output_folder}/table_persective_img.png", table_persective_img)
cv2.imwrite(f"{output_folder}/table_original_persective_img.png", table_original_persective_img)
cv2.imwrite(f"{output_folder}/table_dummy_persective_img.png", table_dummy_persective_img)


True

## ตารางเกรด

### หา column ของตาราง

In [68]:
# ฟังก์ชันสำหรับรวมกลุ่ม indices ที่ติดกันเข้าด้วยกัน
def group_indices(indices, gap=1):
    groups = []
    if not indices:
        return groups
    current = [indices[0]]
    for idx in indices[1:]:
        if idx - current[-1] <= gap:
            current.append(idx)
        else:
            groups.append(current)
            current = [idx]
    groups.append(current)
    return groups

def find_table_columns_rows(table_dummy_persective_img, table_persective_img):

    # คำนวณ horizontal projection (ผลรวมของ pixel ในแต่ละแถว)
    horizontal_proj = np.sum(table_dummy_persective_img, axis=1)
    vertical_proj = np.sum(table_dummy_persective_img, axis=0)

    # ตั้ง threshold สำหรับเลือกเฉพาะลอลัมที่มี "เส้น"
    horizontal_line_threshold = np.max(horizontal_proj) * 0.8
    vertical_line_threshold = np.max(vertical_proj) * 0.7

    # หา indices ของแถวที่มีค่า projection มากกว่าค่า threshold
    row_line_indices = [i for i, value in enumerate(horizontal_proj) if value > horizontal_line_threshold]
    col_line_indices = [i for i, value in enumerate(vertical_proj) if value > vertical_line_threshold]

    # รวมกลุ่ม indices ที่ติดกัน
    groups_row = group_indices(row_line_indices, gap=1)
    groups_col = group_indices(col_line_indices, gap=1)

    # สำหรับแต่ละกลุ่ม หา index กลางเป็นตำแหน่งของเส้น
    # แต่แทนที่จะใช้ความหนาแน่น เราจะใช้ความยาวของแต่ละกลุ่ม (จำนวน index ในกลุ่ม)
    groups_row_with_length = [(group, len(group)) for group in groups_row]
    groups_col_with_length = [(group, len(group)) for group in groups_col]

    # เรียงลำดับกลุ่มจากความยาวมากไปหาน้อย
    groups_row_with_length.sort(key=lambda x: x[1], reverse=True)
    groups_col_with_length.sort(key=lambda x: x[1], reverse=True)

    # เลือกเฉพาะกลุ่มที่มีความยาวมากที่สุด
    top_groups_row = groups_row_with_length[:3]
    top_groups_col = groups_col_with_length[:11]

    # คำนวณตำแหน่งเส้นโดยการหาค่าเฉลี่ยของแต่ละกลุ่ม แล้วเรียงลำดับ
    row_lines = [int(np.mean(group)) for group, _ in top_groups_row]
    col_lines = [int(np.mean(group)) for group, _ in top_groups_col]
    row_lines.sort()
    col_lines.sort()

    print("ตำแหน่งของเส้นแถวที่ตรวจจับได้:", row_lines)
    print("ตำแหน่งของเส้นคอลัมน์ที่ตรวจจับได้:", col_lines)

    mask_row = np.zeros_like(table_dummy_persective_img)
    mask_col = np.zeros_like(table_dummy_persective_img)

    # วาดเส้นแถวลงใน mask (เส้นแนวนอน)
    for y in row_lines:
        cv2.line(mask_row, (0, y), (mask_row.shape[1]-1, y), 255, thickness=10)
    cv2.imwrite(f"{output_folder}/table/row_lines_mask.png", mask_row)

    # วาดเส้นคอลัมน์ลงใน mask (เส้นแนวตั้ง)
    for x in col_lines:
        cv2.line(mask_col, (x, 0), (x, mask_col.shape[0]-1), 255, thickness=10)
    cv2.imwrite(f"{output_folder}/table/col_lines_mask.png", mask_col)

    # --- ใช้ cv2.bitwise_and เพื่อลบเส้นแถวออกจากภาพ ---
    mask_row_inv = cv2.bitwise_not(mask_row)
    img_no_lines_row = cv2.bitwise_and(table_persective_img, table_persective_img, mask=mask_row_inv)
    cv2.imwrite(f"{output_folder}/table/table_no_lines_bitwise_row.png", img_no_lines_row)

    # --- ใช้ cv2.bitwise_and เพื่อลบเส้นแนวตั้งออกจากภาพ ---
    mask_col_inv = cv2.bitwise_not(mask_col)
    img_no_lines_col = cv2.bitwise_and(table_persective_img, table_persective_img, mask=mask_col_inv)
    cv2.imwrite(f"{output_folder}/table/table_no_lines_bitwise_col.png", img_no_lines_col)

    # --- ใช้ cv2.bitwise_and เพื่อลบเส้นแนวตั้งและแนวนอนออกจากภาพ ---
    combined_mask = cv2.bitwise_or(mask_row, mask_col)
    combined_mask_inv = cv2.bitwise_not(combined_mask)
    img_no_lines_row_col = cv2.bitwise_and(table_persective_img, table_persective_img, mask=combined_mask_inv)

    cv2.imwrite(f"{output_folder}/table/img_no_lines_row_col.png", img_no_lines_row_col)

    cropped_col_segments = []

    for i in range(len(col_lines) - 1):
        x_start = col_lines[i]
        x_end = col_lines[i+1]
        y_start = row_lines[1]
        y_end = row_lines[-1]
        
        cropped = img_no_lines_row_col[y_start:y_end, x_start:x_end]  # crop ทุกคอลัมน์ในช่วงแถวที่กำหนด
        cropped_col_segments.append(cropped)
        cv2.imwrite(f"{output_folder}/table/cropped_segment_{i+1}.png", cropped)

    return cropped_col_segments

cell_images = find_table_columns_rows(table_dummy_persective_img, table_persective_img)

ตำแหน่งของเส้นแถวที่ตรวจจับได้: [3, 188, 1987]
ตำแหน่งของเส้นคอลัมน์ที่ตรวจจับได้: [3, 172, 822, 903, 976, 1051, 1221, 1870, 1951, 2025, 2098]


In [70]:
cell_subject_code_img = cell_images[0]
cell_subject_name_img = cell_images[1]
cell_credit_img = cell_images[2]
cell_academic_results_img = cell_images[3]
cell_subject_code_img_2 = cell_images[5]
cell_subject_name_img_2 = cell_images[6]
cell_credit_img_2 = cell_images[7]
cell_academic_results_img_2 = cell_images[8]

'''
plt.figure(figsize=(5,5))
plt.imshow(cell_subject_name_img, cmap="gray")
plt.title(f"binary_image")
plt.show()'
'''


'\nplt.figure(figsize=(5,5))\nplt.imshow(cell_subject_name_img, cmap="gray")\nplt.title(f"binary_image")\nplt.show()\'\n'

### จับกลุ่มข้อความของ cell ตาราง

In [71]:
def detect_text_group_in_cell(cell_img, mode=0, calculate_line_stats=None):
    text_group_images = []

    kernel_open = np.ones((4, 4), np.uint8)
    #kernel_open = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    remove_noise = cv2.morphologyEx(cell_img, cv2.MORPH_OPEN, kernel_open, iterations=1)

    #cv2.imwrite(f"{output_folder}/cell_images/remove_noise.jpg", remove_noise)

    kernel = np.ones((3, 13), np.uint8)
    group_text_img = cv2.dilate(remove_noise, kernel, iterations=2)
    rgb_image = cv2.cvtColor(cell_img.copy(), cv2.COLOR_GRAY2RGB)

    #plt.figure(figsize=(15, 15))
    #plt.imshow(group_text, cmap="gray")

    if(mode == 1):
        # ใช้ Connected Component Analysis
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(group_text_img, connectivity=8)

        text_stats = stats[1:]
        sorted_indices = np.argsort(text_stats[:, 1])  # จัดเรียงตามค่า y (คอลัมน์ที่ 1)
        sorted_stats = text_stats[sorted_indices]

        # ใช้ Boolean Indexing เพื่อเอา noise ออก 
        sorted_stats = sorted_stats[sorted_stats[:, 4] >= 2000]
        calculate_line_stats = []

        for idx_stat, stat in enumerate(sorted_stats):
            x, y, w, h, area = stat
            #print(f"CCA #{idx_stat}: bounding box = (x={x}, y={y}, w={w}, h={h}, area={area})")
            if idx_stat == (len(sorted_stats)-1):
                #print("เข้าเงื่อนลำดัยสุดท้าย",idx_stat)
                x, y, w, h, area = stat
                new_y = round(y-(h/2))
                new_h = round(h+(h*0.8))
                calculate_line_stats.append([x, new_y, w, new_h, area])
            else:
                current_stat = stat
                next_stat = sorted_stats[idx_stat+1]

                distance = next_stat[1] - current_stat[1]
                line_spacing = distance/current_stat[3]

                if line_spacing > 3 and line_spacing <= 5: # เป็นชื่อวิชาที่มีความยาวมากกว่า 1 บรรทัด
                    #print("เข้าเงื่อนไข มากกว่า 1 บรรทัด")
                    x, y, w, h, area = current_stat
                    new_y = round(y-(h/2))
                    new_h = round(h+(h*2.5))
                    calculate_line_stats.append([x, new_y, w, new_h, area])

                elif line_spacing > 6: # เป็นช่องว่างที่ไม่มีวิชา
                    #print("เข้าเงื่อนไข เป็นช่องว่างที่ไม่มีวิชา")
                    x, y, w, h, area = current_stat
                    new_y = round(y-(h/2))
                    new_h = round(h+h)
                    calculate_line_stats.append([x, new_y, w, new_h, area])
                
                else: # เป็นชื่อวิชาที่มีความยาวแค่ว่า 1 บรรทัด
                    #print("เข้าเงื่อนไข 1 บรรทัด")
                    x, y, w, h, area = current_stat
                    new_y = round(y-(h/2))
                    new_h = round(h+(h*0.8))
                    calculate_line_stats.append([x, new_y, w, new_h, area])

        calculate_line_stats = np.array(calculate_line_stats) 

            #print(f"CCA #{idx_stat}: bounding box = (x={x}, y={y}, w={w}, h={h}, area={area})")
            #cv2.rectangle(rgb_image, (x, y), (x + w, y + h), (0, 255, 0), 1)
        #cv2.imwrite(f"{output_folder}/cell_images/cca.jpg", rgb_image)

    text_stats = sorted_stats if mode == 1 else calculate_line_stats

    
    for idx, stats in enumerate(text_stats): # เก็บภาพกลุม
        x, y, w, h, area = stats
   
        if mode == 1:
            cca_img = cell_img[y:y+h, x:x+w]
        if mode == 2:
            #print(f"stats #{idx}: bounding box = (x={x}, y={y}, w={w}, h={h}, area={area})")
            if idx == 0: # ดักบัค crop รูปเกินขอบเขต
                cca_img = cell_img[y+5:y+h, :]
            elif idx == (len(text_stats)-1):
                cca_img = cell_img[y:y+h-5, :]
            else:
                cca_img = cell_img[y:y+h, :]
        text_group_images.append(cca_img)

        # หาขนาดของภาพ (ความกว้างและความสูง)
        image_height, image_width, _ = rgb_image.shape  # ได้ค่า (สูง, กว้าง, ช่องสี)
        cv2.rectangle(rgb_image, (x, y), (image_width, y + h), (0, 255, 0), 1)

    if mode == 1:
        return text_group_images, calculate_line_stats, rgb_image
    else:
        return text_group_images, rgb_image

# ตารางครึ่งแรก
text_subject_code_images, calculate_line_stats_1, subject_code_img = detect_text_group_in_cell(cell_subject_code_img, 1)
text_subject_name_images, subject_name_img = detect_text_group_in_cell(cell_subject_name_img, 2, calculate_line_stats_1)
text_credit_images, credit_img = detect_text_group_in_cell(cell_credit_img, 2, calculate_line_stats_1)
text_academic_results_images, academic_results_img = detect_text_group_in_cell(cell_academic_results_img, 2, calculate_line_stats_1)

# ตารางครึ่งหลัง
text_subject_code_images_2, calculate_line_stats_2, subject_code_img_2 = detect_text_group_in_cell(cell_subject_code_img_2, 1)
text_subject_name_images_2, subject_name_img_2 = detect_text_group_in_cell(cell_subject_name_img_2, 2, calculate_line_stats_2)
text_credit_images_2, credit_img_2 = detect_text_group_in_cell(cell_credit_img_2, 2, calculate_line_stats_2)
text_academic_results_images_2, academic_results_img_2 = detect_text_group_in_cell(cell_academic_results_img_2, 2, calculate_line_stats_2)

cv2.imwrite(f"{output_folder}/cell_images/cca_subject_code_img.jpg", subject_code_img)
cv2.imwrite(f"{output_folder}/cell_images/cca_subject_name_img.jpg", subject_name_img)
cv2.imwrite(f"{output_folder}/cell_images/cca_credit_img.jpg", credit_img)
cv2.imwrite(f"{output_folder}/cell_images/cca_academic_results_img.jpg", academic_results_img)
cv2.imwrite(f"{output_folder}/cell_images/cca_subject_code_img_2.jpg", subject_code_img_2)
cv2.imwrite(f"{output_folder}/cell_images/cca_subject_name_img_2.jpg", subject_name_img_2)
cv2.imwrite(f"{output_folder}/cell_images/cca_credit_img_2.jpg", credit_img_2)
cv2.imwrite(f"{output_folder}/cell_images/cca_academic_results_img_2.jpg", academic_results_img_2)
#cv2.imwrite(f"{output_folder}/cell_images/cca_text_subject_name_images.jpg", subject_name_img)


'''
for idx, img in enumerate(text_subject_name_images):
    print(idx)
    plt.figure(figsize=(15, 15))
    plt.imshow(img, cmap="gray")
'''

'\nfor idx, img in enumerate(text_subject_name_images):\n    print(idx)\n    plt.figure(figsize=(15, 15))\n    plt.imshow(img, cmap="gray")\n'

In [ ]:
# ทดสอบดูรูป
for idx_g, text_group in enumerate(text_subject_code_images):
    plt.figure(figsize=(4,4))
    plt.imshow(text_group, cmap="gray")
    plt.title(f"sub text {idx_g+1}")
    plt.show()

### จับข้อความย่อยในกลุ่มข้อความของ cell ตาราง

In [72]:
def detect_sub_text_in_group(binary_image):

    debug = False
    text_group = []
    for idx, img in enumerate(binary_image):

        if debug == True:
            plt.figure(figsize=(5,5))
            plt.imshow(img, cmap="gray")
            plt.title(f"binary_image")
            plt.show()

        sub_text_images = []

        kernel_open = np.ones((3, 3), np.uint8)
        remove_noise = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel_open, iterations=1)

        if debug == True:
            plt.figure(figsize=(5,5))
            plt.imshow(remove_noise, cmap="gray")
            plt.title(f"remove_noise")
            plt.show()

        # เช็คว่าภาพเป็นสีดำทั้งหมดหรือไม่
        if not np.any(remove_noise):  # ถ้าค่าพิกเซลทั้งหมดเป็น 0 (ดำสนิท)
            print("ภาพเป็นสีดำทั้งหมด")
            sub_text_images.append(remove_noise)
            #return sub_text_images 
        
        else:
            kernel = np.ones((6, 6), np.uint8)
            dummy_image = cv2.dilate(remove_noise, kernel, iterations=2)

            if debug == True:
                plt.figure(figsize=(5,5))
                plt.imshow(dummy_image, cmap="gray")
                plt.title(f"dummy_image")
                plt.show()

            # ใช้ Connected Component Analysis
            num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(dummy_image, connectivity=8)
            char_stats = stats[1:] # ข้าม Background (index 0)
            sorted_indices = np.argsort(char_stats[:, 0]) # จัดเรียงตามค่า x (คอลัมน์ที่ 0)
            sorted_stats = char_stats[sorted_indices]

            # ใช้ Boolean Indexing เพื่อเอา noise ออก 
            sorted_stats = sorted_stats[sorted_stats[:, 4] >= 200]

            for idx, stats in enumerate(sorted_stats):
                #x, y, w, h, area = stats[i]
                x, y, w, h, area = stats

                crop_img = img[y:y+h, x:x+w]
                sub_text_images.append(crop_img)

            if debug == True: 
                plt.figure(figsize=(3,3))
                plt.imshow(crop_img, cmap="gray")
                plt.title(f"crop_img")
                plt.show()

        text_group.append(sub_text_images)

    return text_group
    
text_group_subject_code = detect_sub_text_in_group(text_subject_code_images)
text_group_subject_name = detect_sub_text_in_group(text_subject_name_images)
text_group_credit = detect_sub_text_in_group(text_credit_images)
text_group_academic_results = detect_sub_text_in_group(text_academic_results_images)

text_group_subject_code_2 = detect_sub_text_in_group(text_subject_code_images_2)
text_group_subject_name_2 = detect_sub_text_in_group(text_subject_name_images_2)
text_group_credit_2 = detect_sub_text_in_group(text_credit_images_2)
text_group_academic_results_2 = detect_sub_text_in_group(text_academic_results_images_2)


ภาพเป็นสีดำทั้งหมด
ภาพเป็นสีดำทั้งหมด
ภาพเป็นสีดำทั้งหมด
ภาพเป็นสีดำทั้งหมด


In [ ]:
# ทดสอบดูรูป
for idx_g, text_group in enumerate(text_group_subject_code):
    print(f"text {idx_g+1}")
    for idx_s, sub_text in enumerate(text_group):
        print(f"sub_text {idx_s+1}")
        plt.figure(figsize=(4,4))
        plt.imshow(sub_text, cmap="gray")
        plt.title(f"sub text {idx_s+1}")
        plt.show()

## การทำนาย

In [73]:
def predict_text(text_group, mode=0):
    
    if mode == 1:
        custom_config = r'--oem 3 --psm 7 -c tessedit_char_whitelist=0123456789-'
    elif mode == 3:
        custom_config = r'--oem 3 --psm 7 -c tessedit_char_whitelist=0123456789.'
    else:
        custom_config = r'--oem 3 --psm 7'

    text_box = []
    for idx_g, text_g in enumerate(text_group):
        sub_text_group = ""
        for idx_sub, sub_text in enumerate(text_g):
            if not np.any(sub_text):
                print("เข้าเงื่อนไข")
                text = "-"
            else:
                text = pytesseract.image_to_string(sub_text, config=custom_config, lang='tha+eng')
            #plt.figure(figsize=(4,4))
            #plt.imshow(sub_text, cmap="gray")
            #plt.title(f"sub text {idx_sub+1}")
            #plt.show()

            sub_text_group += text + " "
            text_cleaned = sub_text_group.replace("\n", "")  # ลบ \n ออก
        text_box.append(text_cleaned)

    return text_box
    
text_box_subject_code = predict_text(text_group_subject_code, 1)
text_box_subject_name = predict_text(text_group_subject_name)
text_box_credit = predict_text(text_group_credit, 3)
text_box_academic_results = predict_text(text_group_academic_results, 0)
text_box_subject_code_2 = predict_text(text_group_subject_code_2, 1)
text_box_subject_name_2 = predict_text(text_group_subject_name_2)
text_box_credit_2 = predict_text(text_group_credit_2, 3)
text_box_academic_results_2 = predict_text(text_group_academic_results_2, 0)



เข้าเงื่อนไข
เข้าเงื่อนไข
เข้าเงื่อนไข
เข้าเงื่อนไข


In [74]:
def show_information(array):
    for idx, data in enumerate(array):
        print(data)


show_information(text_box_subject_code)

show_information(text_box_subject_name)

show_information(text_box_credit)

show_information(text_box_academic_results)

show_information(text_box_subject_code_2)

show_information(text_box_subject_name_2)

show_information(text_box_credit_2)

show_information(text_box_academic_results_2)

230000-1101 
30000-1201 
30000-1501 
30000-2001 
30001-1002 
30200-1001 
30201-2001 
30201-2401 
30201-2102 
30201-9006 
40000-1262 
30000-1219 
30000-1605 
30000-1608 
30000-2002 
30001-1055 
30001-2003 
30200-1002 
30201-2002 
30201-2003 
30201-2005 
30000-1308 
ทักษะภาษาไทยเชิงวิชาชีพ 
ภาษาอังกฤษเพื่อการลือสาร 
ชีวิตกับสังคมไทย 
กิจกรรมองค์การวิชาชีพ 1 
องค์การและการจัดการสมัยใหม่ 
เน 
การบัญชีชั้นกลาง 1 
ระบบบัญชี 
โปรแกรมสําเร็จรูปเพื่องานบัญชี 
การบัญชีการเงิน 
ภาษาอังกฤษสําหรับการปฏิบัติงาน 
การสนทนาภาษาจีนเพื่อการทํางาน 
มนุษยสัมพันธ์ในการทํางาน 
การออกกําลังกายเพื่อสุขภาพ 
กิจกรรมองค์การวิชาชีพ 2 
กฎหมายธุรกิจ 
เทคโนโลยีดิจิทัลเพื่อการจัดการอาชีพ 
หลักการตลาด 
การบัญขีขั้นกลาง 2 
การบัญชีต้นทุน 1 
การบัญชีขันสูง 1 
วิทยาศาสตร์งานธุรกิจและบริการ 
3 
3 
3 
- 
3 
3 
3 
3 
3 
3 
2 
2 
2 
1 
- 
3 
3 
3 
3 
3 
3 
3 
4.0 
3.5 
4.0 
. 
4.0 
4.0 
15 
4.0 
3.5 
4.0 
40 
3.0 
4.0 
4.0 
. 
4.0 
4.0 
40 
4.0 
4.0 
40 
3.5 
30000-1401 
30000-2003 
30201-2004 
30201-2006 
30201-2007 
30201-2

## ข้อมูลนักศึกษา

In [77]:
student_info_fh_img, student_info_sh_img

def crop_border(image, left_percent=0, right_percent=0, top_percent=0, bottom_percent=0):
    
    # หาความกว้างและความสูงของภาพ
    height, width = image.shape
    print(height)
    print(width)

    # คำนวณพิกัดที่จะตัด (แปลงเป็นพิกเซล)
    x_start = int(width * left_percent)
    x_end = int(width * (1 - right_percent))
    y_start = int(height * top_percent)
    y_end = int(height * (1 - bottom_percent))

    # ตัดภาพ (Crop)
    cropped_img = image[y_start:y_end, x_start:x_end]

    #cv2.imwrite(f"{output_folder}/cropped_fh.jpg", cropped_img)
    
    return cropped_img

def find_text_student_info_fh(student_info_fh_img):
    student_info_fh_img = crop_border(student_info_fh_img.copy(), 0.06, 0.06, 0.06, 0.01)

    rgb_image = cv2.cvtColor(student_info_fh_img.copy(), cv2.COLOR_GRAY2RGB)
    
    # กำหนด kernel (ขนาดของ kernel สามารถปรับเปลี่ยนได้ตามความเหมาะสม)
    kernel_open = np.ones((2, 2), np.uint8)
    kernel_close = np.ones((6, 50), np.uint8)
    
    opening = cv2.morphologyEx(student_info_fh_img.copy(), cv2.MORPH_OPEN, kernel=kernel_open, iterations=1)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel=kernel_close, iterations=2)

    rgb_closing_image = cv2.cvtColor(closing, cv2.COLOR_GRAY2RGB)

    cv2.imwrite(f"{output_folder}/opening.jpg", opening)
    cv2.imwrite(f"{output_folder}/closing.jpg", closing)

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closing, connectivity=8)

    # 1. เอา stats ตัวแรก (background) ออก
    stats_no_bg = stats[1:]

    # 2. เรียง stats ใหม่โดยใช้ค่า area (คอลัมน์ที่ 4) จากมากไปน้อย
    sorted_indices = np.argsort(-stats_no_bg[:, 4])
    sorted_stats = stats_no_bg[sorted_indices]

    # 3. เลือกแค่ 13 element ที่มีค่า area สูงสุด
    top_13_stats = sorted_stats[:13]
    top_13_stats_sorted_by_y = top_13_stats[np.argsort(top_13_stats[:, 1])]

    img_height, img_width = student_info_fh_img.shape[:2]

    # กำหนด margin เป็นเปอร์เซ็นต์ของขนาด bounding box
    # เช่น กำหนด 10% ของความกว้าง/ความสูงของ bounding box สำหรับแต่ละด้าน
    left_margin_percent = 0.1    # ขยายซ้าย 10%
    right_margin_percent = 0.1   # ขยายขวา 10%
    top_margin_percent = 0.2     # ขยายบน 20%
    bottom_margin_percent = 0.1  # ขยายล่าง 10%

    text_group_stud_fh = []
    for idx, stats in enumerate(top_13_stats_sorted_by_y): # เก็บภาพกลุม
        x, y, w, h, area = stats

        # คำนวณ margin ตามเปอร์เซ็นต์ของ bounding box
        left_margin = int(w * left_margin_percent)
        right_margin = int(w * right_margin_percent)
        top_margin = int(h * top_margin_percent)
        bottom_margin = int(h * bottom_margin_percent)

        # คำนวณพิกัดใหม่โดยใช้ margin ที่คำนวณได้
        x_new = max(x - left_margin, 0)
        y_new = max(y - top_margin, 0)
        x_end = min(x + w + right_margin, img_width)
        y_end = min(y + h + bottom_margin, img_height)

        cluster_img = student_info_fh_img[y_new:y_end, x_new:x_end]
        text_group_stud_fh.append(cluster_img)

        # วาดกรอบที่ขยายแล้วลงบนภาพ
        cv2.rectangle(rgb_image, (x_new, y_new), (x_end, y_end), (0, 255, 0), 1)
        cv2.rectangle(rgb_closing_image, (x_new, y_new), (x_end, y_end), (0, 255, 0), 1)
        
    cv2.imwrite(f"{output_folder}/cca_top_13_stats.jpg", rgb_image)
    cv2.imwrite(f"{output_folder}/cca_rgb_closing_image.jpg", rgb_closing_image)

    return text_group_stud_fh[1:]

def find_text_student_info_sh(student_info_sh_img):
    student_info_sh_img = crop_border(student_info_sh_img.copy(), 0.05, 0.00, 0.05, 0.01)

    rgb_image = cv2.cvtColor(student_info_sh_img.copy(), cv2.COLOR_GRAY2RGB)
    
    # กำหนด kernel (ขนาดของ kernel สามารถปรับเปลี่ยนได้ตามความเหมาะสม)
    kernel_open = np.ones((2, 2), np.uint8)
    kernel_close = np.ones((8, 50), np.uint8)
    
    opening = cv2.morphologyEx(student_info_sh_img.copy(), cv2.MORPH_OPEN, kernel=kernel_open, iterations=1)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel=kernel_close, iterations=2)

    rgb_closing_image = cv2.cvtColor(closing, cv2.COLOR_GRAY2RGB)

    cv2.imwrite(f"{output_folder}/opening_sh.jpg", opening)
    cv2.imwrite(f"{output_folder}/closing_sh.jpg", closing)

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closing, connectivity=8)

    # 1. เอา stats ตัวแรก (background) ออก
    stats_no_bg = stats[1:]

    # 2. เรียง stats ใหม่โดยใช้ค่า area (คอลัมน์ที่ 4) จากมากไปน้อย
    sorted_indices = np.argsort(-stats_no_bg[:, 4])
    sorted_stats = stats_no_bg[sorted_indices]

    # 3. เลือกแค่ 14 element ที่มีค่า area สูงสุด
    top_14_stats = sorted_stats[:14]
    top_14_stats_sorted_by_y = top_14_stats[np.argsort(top_14_stats[:, 1])]

    img_height, img_width = student_info_sh_img.shape[:2]

    # กำหนด margin เป็นเปอร์เซ็นต์ของขนาด bounding box
    # เช่น กำหนด 10% ของความกว้าง/ความสูงของ bounding box สำหรับแต่ละด้าน
    left_margin_percent = 0.1    # ขยายซ้าย 10%
    right_margin_percent = 0.1   # ขยายขวา 10%
    top_margin_percent = 0.2     # ขยายบน 20%
    bottom_margin_percent = 0.1  # ขยายล่าง 10%

    text_group_stud_sh = []
    for idx, stats in enumerate(top_14_stats_sorted_by_y): # เก็บภาพกลุม
        x, y, w, h, area = stats

        # คำนวณ margin ตามเปอร์เซ็นต์ของ bounding box
        left_margin = int(w * left_margin_percent)
        right_margin = int(w * right_margin_percent)
        top_margin = int(h * top_margin_percent)
        bottom_margin = int(h * bottom_margin_percent)

        # คำนวณพิกัดใหม่โดยใช้ margin ที่คำนวณได้
        x_new = max(x - left_margin, 0)
        y_new = max(y - top_margin, 0)
        x_end = min(x + w + right_margin, img_width)
        y_end = min(y + h + bottom_margin, img_height)

        cluster_img = student_info_sh_img[y_new:y_end, x_new:x_end]
        text_group_stud_sh.append(cluster_img)

        # วาดกรอบที่ขยายแล้วลงบนภาพ
        cv2.rectangle(rgb_image, (x_new, y_new), (x_end, y_end), (0, 255, 0), 1)
        cv2.rectangle(rgb_closing_image, (x_new, y_new), (x_end, y_end), (0, 255, 0), 1)
        
    cv2.imwrite(f"{output_folder}/cca_top_14_stats.jpg", rgb_image)
    cv2.imwrite(f"{output_folder}/cca_rgb_closing_image.jpg", rgb_closing_image)

    return text_group_stud_sh[3:]

text_stud_fh_images = find_text_student_info_fh(student_info_fh_img)
text_stud_sh_images = find_text_student_info_sh(student_info_sh_img)

742
587
742
727


In [ ]:
# ทดสอบดูรูป
for idx_g, text_group in enumerate(text_stud_fh_images):
    print(f"text {idx_g+1}")
    plt.figure(figsize=(4,4))
    plt.imshow(text_group, cmap="gray")
    plt.title(f"text_group {idx_g+1}")
    plt.show()

In [78]:
indices_fh = [3, -2, -1]
indices_sh = [-3, -1]
student_name, field_of_study, field_of_work = [text_stud_fh_images[i] for i in indices_fh]
gpa, graduation_date = [text_stud_sh_images[i] for i in indices_sh]


'''
plt.figure(figsize=(4,4))
plt.imshow(field_of_work, cmap="gray")
plt.title(f"student_name")
plt.show()
'''


'\nplt.figure(figsize=(4,4))\nplt.imshow(field_of_work, cmap="gray")\nplt.title(f"student_name")\nplt.show()\n'

### จับกลุ่มข้อความย่อย

In [79]:
def detect_sub_text_in_group_stud(binary_image):
    
    '''
    plt.figure(figsize=(5,5))
    plt.imshow(binary_image, cmap="gray")
    plt.title(f"binary_image")
    plt.show()
    '''
    text_group = []

    #kernel_open = np.ones((2, 2), np.uint8)
    #remove_noise = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel_open, iterations=1)

    #plt.figure(figsize=(5,5))
    #plt.imshow(remove_noise, cmap="gray")
    #plt.title(f"remove_noise")
    #plt.show()

    kernel = np.ones((3, 3), np.uint8)
    dummy_image = cv2.dilate(binary_image, kernel, iterations=2)

    '''
    plt.figure(figsize=(5,5))
    plt.imshow(dummy_image, cmap="gray")
    plt.title(f"dummy_image")
    plt.show()
    '''

    # ใช้ Connected Component Analysis
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(dummy_image, connectivity=8)
    word_stats = stats[1:] # ข้าม Background (index 0)
    sorted_indices = np.argsort(word_stats[:, 0]) # จัดเรียงตามค่า x (คอลัมน์ที่ 0)
    sorted_stats = word_stats[sorted_indices]

    for idx, stats in enumerate(sorted_stats):
        x, y, w, h, area = stats
        cluster_img = binary_image[y:y+h, x:x+w]
        text_group.append(cluster_img)

    return text_group

text_group_student_name = detect_sub_text_in_group_stud(student_name)
text_group_field_of_study = detect_sub_text_in_group_stud(field_of_study)
text_group_field_of_work = detect_sub_text_in_group_stud(field_of_work)

text_group_gpa = detect_sub_text_in_group_stud(gpa)
text_group_graduation_date = detect_sub_text_in_group_stud(graduation_date)


In [ ]:
# ทดสอบดูรูป
for idx, sub_text in enumerate(text_group_gpa):
    print(f"text {idx_g+1}")
    plt.figure(figsize=(4,4))
    plt.imshow(sub_text, cmap="gray")
    plt.title(f"sub_text {idx+1}")
    plt.show()

### ทำนาย

In [80]:
def predict_text_stud(text_group, mode=0):
    
    custom_config = r'--oem 3 --psm 7'

    sub_text_group = ""
    for idx_sub, sub_text in enumerate(text_group):
        if not np.any(sub_text):
            print("เข้าเงื่อนไข")
            text = "-"
        else:
            text = pytesseract.image_to_string(sub_text, config=custom_config, lang='tha')
        #plt.figure(figsize=(4,4))
        #plt.imshow(sub_text, cmap="gray")
        #plt.title(f"sub text {idx_sub+1}")
        #plt.show()

        sub_text_group += text + " "
        text_cleaned = sub_text_group.replace("\n", "")  # ลบ \n ออก

    return text_cleaned

text_box_student_name = predict_text_stud(text_group_student_name[3:], 1)
text_box_field_of_study = predict_text_stud(text_group_field_of_study[1:], 1)
text_box_field_of_work = predict_text_stud(text_group_field_of_work[1:], 1)

text_box_gpa = predict_text_stud(text_group_gpa[1:], 1)
text_box_graduation_date = predict_text_stud(text_group_graduation_date[1:], 1)

print(text_box_student_name)
print(text_box_field_of_study)
print(text_box_field_of_work)

print(text_box_gpa)
print(text_box_graduation_date)

นางสาวจิรนันท์ พีรวศินสกุล 
การบัญชี 
การบัญชี 
3, 76 
23 มีนาคม 2565 


# JSON

In [83]:
import json

# ทำการรวม list
text_box_subject_code_all = text_box_subject_code + text_box_subject_code_2
text_box_subject_name_all = text_box_subject_name + text_box_subject_name_2
text_box_credit_all = text_box_credit + text_box_credit_2
text_box_academic_results_all = text_box_academic_results + text_box_academic_results_2

# ลบช่องว่างด้วย list comprehension
subject_codes = [scode.replace(" ", "") for scode in text_box_subject_code_all]
subject_names = [sname.strip() for sname in text_box_subject_name_all]
credits = [credit.replace(" ", "") for credit in text_box_credit_all] 
grades = [grade.replace(" ", "") for grade in text_box_academic_results_all]

student_name = text_box_student_name.strip()
field_of_study = text_box_field_of_study.replace(" ", "")
field_of_work = text_box_field_of_work.replace(" ", "")
gpa = text_box_gpa.replace(" ", "")
graduation_date = text_box_graduation_date.strip()

enrolled_subjects = []
for scode, sname, credit, grade in zip(subject_codes, subject_names, credits, grades):
    enrolled_subjects.append({
        "subject_code": scode,
        "subject_name": sname,
        "credit": credit,
        "grade": grade
    })

# สร้างโครงสร้าง JSON ในรูปแบบของ dictionary
data = {
    "student_info": {
        "fullname": student_name,
        "field_of_study": field_of_study,
        "field_of_work": field_of_work,
        "gpa": gpa,
        "graduation_date":graduation_date
    },
    "enrolled_subjects": enrolled_subjects
}

# แปลง Python dictionary เป็น JSON string
json_string = json.dumps(data, indent=4, ensure_ascii=False)
print(json_string)

{
    "student_info": {
        "fullname": "นางสาวจิรนันท์ พีรวศินสกุล",
        "field_of_study": "การบัญชี",
        "field_of_work": "การบัญชี",
        "gpa": "3,76",
        "graduation_date": "23 มีนาคม 2565"
    },
    "enrolled_subjects": [
        {
            "subject_code": "230000-1101",
            "subject_name": "ทักษะภาษาไทยเชิงวิชาชีพ",
            "credit": "3",
            "grade": "4.0"
        },
        {
            "subject_code": "30000-1201",
            "subject_name": "ภาษาอังกฤษเพื่อการลือสาร",
            "credit": "3",
            "grade": "3.5"
        },
        {
            "subject_code": "30000-1501",
            "subject_name": "ชีวิตกับสังคมไทย",
            "credit": "3",
            "grade": "4.0"
        },
        {
            "subject_code": "30000-2001",
            "subject_name": "กิจกรรมองค์การวิชาชีพ 1",
            "credit": "-",
            "grade": "."
        },
        {
            "subject_code": "30001-1002",
            "subj